<img align="left" src="https://panoptes-uploads.zooniverse.org/project_avatar/86c23ca7-bbaa-4e84-8d8a-876819551431.png" type="image/png" height=100 width=100>
</img>


<h1 align="right">Colab KSO Tutorials #8: Analyse Zooniverse classifications</h1>
<h3 align="right">Written by @jannesgg and @vykanton</h3>
<h5 align="right">Last updated: April 11, 2022</h5>

# Set up and requirements

In [ ]:
!git clone --recurse-submodules -b dev https://github.com/ocean-data-factory-sweden/koster_data_management.git
!pip install -r koster_data_management/requirements.txt

# Deal with github installed packages
import site
site.main()

### Import Python packages

In [ ]:
# Set the directory of the libraries
import sys, os
os.chdir("koster_data_management/tutorials")
sys.path.append('..')

# Import required modules
import kso_utils.tutorials_utils as t_utils
import kso_utils.project_utils as p_utils
import kso_utils.t8_utils as t8

print("Packages loaded successfully")

### Choose your project

In [4]:
project_name = t_utils.choose_project()

Dropdown(description='Project:', options=('Koster_Seafloor_Obs', 'Spyfish_Aotearoa', 'SGU', 'Medins'), value='…

In [5]:
project = p_utils.find_project(project_name=project_name.value)

### Set up initial information

In [ ]:
db_info_dict, zoo_project, zoo_info_dict = t8.setup_initial_info(project)

### Step 1: Specify the Zooniverse workflow id and version of interest

*Note:  A manual export in Zooniverse is required to get the most up-to-date classifications here**

Make sure your workflows in Zooniverse have different names to avoid issues while selecting the workflow id

In [7]:
# Display a selectable list of workflow names and a list of versions of the workflow of interest
workflows_df = zoo_info_dict["workflows"]
wm = t8.WidgetMaker(workflows_df)
wm

WidgetMaker(children=(IntText(value=0, description='Number of workflows:', style=DescriptionStyle(description_…

Output()

In [8]:
# Retrieve classifications from the workflow of interest
class_df = t8.get_classifications(wm.checks,
                                   workflows_df,
                                   wm.checks['Subject type: #0'], 
                                   zoo_info_dict["classifications"], 
                                   db_info_dict["db_path"],
                                   project)

Zooniverse classifications have been retrieved


### Step 2: Aggregate classifications received on the workflow of interest

In [9]:
# Specify the agreement threshold required among cit scientists
agg_params = t8.choose_agg_parameters(wm.checks['Subject type: #0'])

FloatSlider(value=0.8, continuous_update=False, description='Aggregation threshold:', max=1.0, readout_format=…

IntSlider(value=3, continuous_update=False, description='Min numbers of users:', max=15, min=1, style=SliderSt…

In [10]:
agg_class_df, raw_class_df = t8.aggregrate_classifications(class_df, 
                                                            wm.checks['Subject type: #0'], 
                                                            project, 
                                                            agg_params)

Aggregrating the classifications
886 classifications aggregated out of 1260 unique subjects available


### Step 3: Summarise the number of classifications based on the agreement specified

In [11]:
agg_class_df.groupby("label")["subject_ids"].agg("count")

,subject_ids
label,


### Step 4: Display the aggregated classifications in a table

In [12]:
# Display the dataframe into a table
t8.launch_table(agg_class_df, wm.checks['Subject type: #0'])

subject_ids,label,how_many,first_seen


### Step 5: Use the subject explorer widget to visualise subjects and their aggregated classifications

In [13]:
# Launch the subject viewer
t8.launch_viewer(agg_class_df, wm.checks['Subject type: #0'])

Combobox(value='', description='Subject id:', ensure_option=True, options=('64027019', '64027020', '64027021',…

Output()

### Step 6: Use the subject explorer widget to get more information about specific subjects and their "raw" classifications

In [ ]:
# Launch the classifications_per_subject explorer
t8.explore_classifications_per_subject(raw_class_df, wm.checks['Subject type: #0'])

In [ ]:
# END